Import Dependencies

In [21]:
import pandas as pd

In [22]:
from bs4 import BeautifulSoup as bs

In [23]:
from selenium import webdriver  

In [26]:
from splinter import Browser

#pointing to the directory where chromedriver exists
#executable_path = {"executable_path":"C:\Users\mebader.NORTHAMERICA\chromedriver_win32\chromedriver"}
executable_path = {"executable_path":"C:/Users/mebader.NORTHAMERICA/chromedriver_win32/chromedriver"}
browser = Browser("chrome", **executable_path, headless = False)
                                   

In [27]:
import requests

Open URLs and Get Info

In [28]:
mars_news = {}
mars_para = []

In [29]:
base_url = "https://mars.nasa.gov/" 
news_url = "https://mars.nasa.gov/news"
resp_1 = requests.get(news_url)                                             

mars_soup = bs(resp_1.text, 'html.parser') 

In [30]:
class_div = mars_soup.find(class_="slide")                                  
marssoup_news = class_div.find_all('a')                                           
marsnews_title = marssoup_news[1].get_text().strip()  

In [31]:
soup_p = class_div.find_all('a', href=True)                                 
soup_p_url = soup_p[0]['href']                                               
paragraph_url = base_url + soup_p_url                                        
response_2 = requests.get(paragraph_url)                                          
paragraph_soup = bs(response_2.text, "html.parser")                               
mars_paragraphs = paragraph_soup.find(class_='wysiwyg_content')                     
paragraphs = mars_paragraphs.find_all('p')

In [32]:
for paragraph in paragraphs:                                                 
    simple_paragraph = paragraph.get_text().strip()                              
    mars_para.append(simple_paragraph)   

In [33]:
mars_news["news_title"] = marsnews_title
mars_news["paragraph_1"] = mars_para[0]
mars_news["paragraph_2"] = mars_para[1]

In [34]:
mars_news
#not sure why I have duplicate title...need to figure out....

{'news_title': 'NASA Invites Students to Name Mars 2020 Rover',
 'paragraph_1': 'Red rover, red rover, send a name for Mars 2020 right over! NASA is recruiting help from students nationwide to find a name for its next Mars rover mission.',
 'paragraph_2': 'Starting Tuesday, Aug. 27, K-12 students in U.S. public, private and home schools can enter the Mars 2020 Name the Rover essay contest. One grand prize winner will name the rover and be invited to see the spacecraft launch in July 2020 from Cape Canaveral Air Force Station in Florida.'}

Now get images from the jpl site to bring in to the web page

In [40]:
browser = Browser('chrome', headless=False) 
#browser = browser('Chrome')
jpl_base_url = 'https://photojournal.jpl.nasa.gov/jpeg/'               
jpl_image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

In [41]:
browser.visit(jpl_image_url)
image_html = browser.html
image_soup = bs(image_html, 'html.parser')

In [39]:
featured_image_list = []
for image in image_soup.find_all('div',class_="img"):
    featured_image_list.append(image.find('img').get('src'))

In [42]:
picture_shown = featured_image_list[0]
remove_size = picture_shown.split('-')
remove_fn = remove_size[0].split('/')
featured_image_url = jpl_base_url + remove_fn[-1] + '.jpg'


In [43]:
featured_image_url

'https://photojournal.jpl.nasa.gov/jpeg/PIA23345.jpg'

In [44]:
browser.quit()

Get Mars Twitter feed

In [51]:
browser = Browser('chrome', headless=False)
twitter_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(twitter_url)

In [52]:
twitter_html = browser.html
twitter_soup = bs(twitter_html, 'html.parser')

In [53]:
mars_weather_list = []
for weather_info in twitter_soup.find_all('p', class_="TweetTextSize--normal js-tweet-text tweet-text"):
    mars_weather_list.append(weather_info.text.strip())

In [56]:
for value in reversed(mars_weather_list):
    if value[:3]=='Sol':
        mars_weather = value

In [57]:
mars_weather

NameError: name 'mars_weather' is not defined